# Model fine-tuning (Llama-2-7b-chat)

Reference: https://www.datacamp.com/tutorial/fine-tuning-llama-2

In [1]:
import os
import torch
import pandas as pd
from datasets import Dataset
from datetime import datetime
from peft import LoraConfig
from tensorboard import notebook
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer

os.chdir("..")

### Load and preprocess training data

In [2]:
train_path = "data/processed/split/counsel-chat-best-answer-train.csv"
df_train = pd.read_csv(train_path).set_index("questionID")
print(df_train.shape)

(445, 6)


In [3]:
df_train.head()

,questionTitle,questionText,topic,answerText,upvotes,views
questionID,,,,,,
1,My apartment manager won't let me keep an emot...,I have been diagnosed with general anxiety and...,depression,"This can be a difficult situation. Typically,...",2,1026
3,Why do I feel like I don't belong anywhere?,There are many people willing to lovingly prov...,depression,I truly understand what you are saying. I want...,1,62
4,How can I help my girlfriend?,My girlfriend just quit drinking and she becam...,depression,You're probably not going to like my answer.Yo...,3,824
10,How do I stop feeling empty?,I don't know how else to explain it. All I can...,depression,Why do I feel empty?Feelings of emptiness—a la...,3,148
12,How can I get my husband to listen to my needs...,"I tried telling my husband I was depressed, an...",depression,"Oh dear.From what you write, your husband does...",1,240


In [4]:
def combine_question_answer(question, answer):
    system_msg = "<<SYS>>\n" \
        + "You are a helpful and honest psychologist." \
        + "Respond to the following question from the user." \
        + "Your answers should not include any critical, dangerous or illegal content." \
        + "Your answers should be unbiased and encouraging.\n" \
        + "<</SYS>>\n\n"
    prompt = f"<s>[INST] {system_msg}###User: {question}###You: [/INST]{answer}</s>"
    return prompt

df_train["text"] = df_train.apply(
    lambda row: combine_question_answer(row.questionText, row.answerText), axis=1
)
df_train[["questionText", "answerText", "text"]].head()

,questionText,answerText,text
questionID,,,
1,I have been diagnosed with general anxiety and...,"This can be a difficult situation. Typically,...",<s>[INST] <<SYS>>\nYou are a helpful and hones...
3,There are many people willing to lovingly prov...,I truly understand what you are saying. I want...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
4,My girlfriend just quit drinking and she becam...,You're probably not going to like my answer.Yo...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
10,I don't know how else to explain it. All I can...,Why do I feel empty?Feelings of emptiness—a la...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
12,"I tried telling my husband I was depressed, an...","Oh dear.From what you write, your husband does...",<s>[INST] <<SYS>>\nYou are a helpful and hones...


In [5]:
df_train = df_train.sample(frac=1)
dataset_train = Dataset.from_pandas(df_train[["text"]][:int(df_train.shape[0]*0.8)])
dataset_eval = Dataset.from_pandas(df_train[["text"]][int(df_train.shape[0]*0.8):])
for text in dataset_train:
    print(text)
    break

{'text': '<s>[INST] <<SYS>>\nYou are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.\n<</SYS>>\n\n###User: I am not sure if I am depressed. I don\'t know how to bring it up to my parents, and that makes me miserable.###You: [/INST]You are not alone, many people fear opening up to family members about the topic of depression or mental illness. There are many different reason why some may fear telling their parents. The most common thoughts I hear in my office are: " My parents won\'t understand me", I may cause more problems to the family", "I am worried that something bad may happen if I tell them".\xa0If possible express your current concerns and worries to your parents. You can start the conversation with your parents by saying "I have not been feeling like myself lately, and I may want to see a counselor".\xa0I think you are do

### Load tokenizer and base model

In [6]:
base_model = "meta-llama/Llama-2-7b-chat-hf"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [8]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

### Set params for PEFT fine-tuning

In [11]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
timestamp = datetime.now().strftime("%y%m%d_%H%M")
training_params = TrainingArguments(
    output_dir=f"./results/results_{timestamp}",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    evaluation_strategy="steps",
    save_steps=10,
    save_total_limit=5,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    load_best_model_at_end=True,
)

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/356 [00:00<?, ? examples/s]

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

### Model fine-tuning

In [14]:
trainer.train()

Step,Training Loss,Validation Loss
10,2.556700,2.469196
20,2.349600,2.118536
30,2.038300,1.894948
40,1.765200,1.851612
50,1.720600,1.831419
60,1.827500,1.822108
70,1.733800,1.816305
80,1.810200,1.809533
90,1.573700,1.808826
100,1.813000,1.804062


TrainOutput(global_step=267, training_loss=1.7556427837757582, metrics={'train_runtime': 3612.157, 'train_samples_per_second': 0.296, 'train_steps_per_second': 0.074, 'total_flos': 1.6067174682476544e+16, 'train_loss': 1.7556427837757582, 'epoch': 3.0})

In [15]:
trainer.model.save_pretrained(f"models/model_{timestamp}")
trainer.tokenizer.save_pretrained(f"models/model_{timestamp}")

('models/model_240411_0952/tokenizer_config.json',
 'models/model_240411_0952/special_tokens_map.json',
 'models/model_240411_0952/tokenizer.model',
 'models/model_240411_0952/added_tokens.json',
 'models/model_240411_0952/tokenizer.json')

### Model inferencing demo

In [16]:
def get_question_prompt(question):
    system_msg = "<<SYS>>\n" \
        + "You are a helpful and honest psychologist." \
        + "Respond to the following question from the user." \
        + "Your answers should not include any critical, dangerous or illegal content." \
        + "Your answers should be unbiased and encouraging.\n" \
        + "<</SYS>>\n\n"
    prompt = f"<s>[INST] {system_msg}###User: {question}###You: [/INST]"
    return prompt

question = "I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
result = pipe(get_question_prompt(question))
print(result[0]['generated_text'])

<s>[INST] <<SYS>>
You are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.
<</SYS>>

###User: I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?###You: [/INST]I think you are on the right track.   You are worried about your school work, and that is a good thing.   You are aware of the potential consequences of not completing your work on time, and that is also a good thing.   I think you should talk to your teacher about your concerns.   They may be able to help you come up with a plan to help you get your work done on time.   Also, you can also try breaking down your work into smaller tasks, and then set deadlines for yourself to complete each one.   You can also try to find a quiet place to study, and try to avoid distractions.   I h

### Visualization

In [4]:
# timestamp = "240411_0952"
# log_dir = f"results/results_{timestamp}/runs"
# notebook.start(f"--logdir {log_dir} --port 4000")